In [1]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [2]:
#get all apps > 10 m tvl
all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
res = res[res['tvl'] > 10_000_000] ##greater than 10mil
print(len(res))
# print(res.columns)

356


In [13]:

protocols = res[['slug','chainTvls']]
re = res['chainTvls']
# r[1].keys()
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
protocols

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_54827/1800157470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )


,slug,chainTvls
0,makerdao,[Ethereum]
1,polygon-bridge-&-staking,[Polygon]
2,lido,"[Ethereum, Moonriver, Solana, Terra, Moonbeam]"
3,uniswap,"[Optimism, Ethereum, Polygon, Celo, Arbitrum]"
4,curve,"[Optimism, Harmony, Moonbeam, Ethereum, Aurora..."
...,...,...
351,88mph,"[Avalanche, Ethereum, Polygon, Ethereum-stakin..."
352,guard(helmet),"[Binance, Polygon]"
353,ouchi-finance,[Ethereum]
354,acala-swap,[Acala]


In [14]:
# protocols = protocols[ protocols['slug'] == 'uniswap-v3' ]
# api_str = 'https://api.llama.fi/protocol/'
# ad = pd.DataFrame( r.get(api_str).json()['chainTvls'] ).T[['tokens']]
# ad

In [15]:
# api_str = 'https://api.llama.fi/protocol/uniswap'
# prot_req = r.get(api_str, headers=header).json()['chainTvls']
# # prot_req['Ethereum']
# prot_req

In [16]:
#get by app
api_str = 'https://api.llama.fi/protocol/'
# print(protocols)
prod = []
for index,proto in protocols.iterrows():
#     print(proto)
    prot = proto['slug']
    chains = proto['chainTvls']
    apic = api_str + prot
#     time.sleep(0.1)
    try:
        prot_req = r.get(apic, headers=header).json()['chainTvls']
        print(apic)
        for ch in chains:
            ad = pd.json_normalize( prot_req[ch]['tokens'] )
            ad_usd = pd.json_normalize( prot_req[ch]['tokensInUsd'] )
#             ad = ad.merge(how='left')
            if not ad.empty:
                ad = pd.melt(ad,id_vars = ['date'])
                ad = ad.rename(columns={'variable':'token','value':'token_value'})
                ad_usd = pd.melt(ad_usd,id_vars = ['date'])
                ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
                ad = ad.merge(ad_usd,on=['date','token'])
                
                ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
                ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
                ad['protocol'] = prot
                ad['chain'] = ch
        #         ad['start_date'] = pd.to_datetime(prot[1])
                ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
                prod.append(ad)
    except:
        print('err')

df_df = pd.concat(prod)
df_df

https://api.llama.fi/protocol/makerdao
https://api.llama.fi/protocol/polygon-bridge-&-staking
https://api.llama.fi/protocol/lido
https://api.llama.fi/protocol/uniswap
https://api.llama.fi/protocol/curve
https://api.llama.fi/protocol/wbtc
https://api.llama.fi/protocol/aave-v2
https://api.llama.fi/protocol/convex-finance
https://api.llama.fi/protocol/justlend
https://api.llama.fi/protocol/pancakeswap
https://api.llama.fi/protocol/compound
https://api.llama.fi/protocol/instadapp
https://api.llama.fi/protocol/multichain
https://api.llama.fi/protocol/arrakis-finance
https://api.llama.fi/protocol/aave-v3
https://api.llama.fi/protocol/balancer
https://api.llama.fi/protocol/justcryptos
https://api.llama.fi/protocol/juststables
https://api.llama.fi/protocol/frax
https://api.llama.fi/protocol/hbtc
https://api.llama.fi/protocol/sunswap
https://api.llama.fi/protocol/sushiswap
https://api.llama.fi/protocol/venus
https://api.llama.fi/protocol/vvs-finance
https://api.llama.fi/protocol/synthetix
https

https://api.llama.fi/protocol/spool-protocol
https://api.llama.fi/protocol/metronome
https://api.llama.fi/protocol/keep-network
https://api.llama.fi/protocol/hashflow
https://api.llama.fi/protocol/harvest-finance
https://api.llama.fi/protocol/hector-network
https://api.llama.fi/protocol/rex-staking
https://api.llama.fi/protocol/xdao
https://api.llama.fi/protocol/liqee
https://api.llama.fi/protocol/thetaswap
https://api.llama.fi/protocol/maple
https://api.llama.fi/protocol/dopex
https://api.llama.fi/protocol/stafi
https://api.llama.fi/protocol/nftx
https://api.llama.fi/protocol/hubble
https://api.llama.fi/protocol/solarbeam
https://api.llama.fi/protocol/filda
https://api.llama.fi/protocol/reserve
https://api.llama.fi/protocol/ooki
https://api.llama.fi/protocol/acryptos
https://api.llama.fi/protocol/unslashed
err
https://api.llama.fi/protocol/astroport
https://api.llama.fi/protocol/tarot
https://api.llama.fi/protocol/iziswap
https://api.llama.fi/protocol/sharedstake
https://api.llama.fi/

,date,token,token_value,usd_value,protocol,chain
0,2022-05-09 00:00:00,MATIC,61211089.17387,5.060529e+07,makerdao,Ethereum
1,2022-05-10 00:00:00,MATIC,61211089.17387,5.437283e+07,makerdao,Ethereum
2,2022-05-11 00:00:00,MATIC,131231089.17387,8.719886e+07,makerdao,Ethereum
3,2022-05-12 00:00:00,MATIC,131264317.97387,7.932946e+07,makerdao,Ethereum
4,2022-05-13 00:00:00,MATIC,131264317.97387,9.167802e+07,makerdao,Ethereum
...,...,...,...,...,...,...
316,2022-08-03 00:00:00,SASHIMI,13090909.07127,9.146631e+04,sashimiswap,staking
317,2022-08-04 00:00:00,SASHIMI,13090909.07127,3.656356e+04,sashimiswap,staking
318,2022-08-05 00:00:00,SASHIMI,13090909.07127,3.665402e+04,sashimiswap,staking
319,2022-08-06 00:00:00,SASHIMI,13090909.07127,3.477535e+04,sashimiswap,staking


In [ ]:
# df_df[df_df['protocol']=='perpetual-protocol']
df_df = df_df[df_df['date'].dt.date >= date.today()-timedelta(days=30) ]
df_df[(df_df['protocol'] == 'uniswap') & (df_df['token'] != 'USDT') & (df_df['token_value'] > 0)]

In [ ]:
# # DISTINCT TOKENS

# token_list = df_df.groupby(['token']).count()
# token_list = token_list[token_list['token_value']>=1000]
# token_list.sort_values(by='token_value',inplace=True, ascending=False)
# token_list.reset_index(inplace=True)
# missing_token_list = token_list[~token_list['token'].isin(cg_token_list)]
# missing_token_list
# missing_token_list

In [ ]:
# data_df = df_df.merge(cg_df, on=['date','token'],how='inner')
data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

# print(len(data_df))
data_df.sort_values(by='date',inplace=True)
# print(len(data_df))
data_df['last_token_value'] = data_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
# data_df['net_token_flow'] = data_df.groupby(['token','protocol','chain'])['token_value'].apply(lambda x: x- x.shift(1))
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']

# print(len(data_df))
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


# data_df = data_df[data_df['token'] != 'RVRS'] #Harmony error
# data_df = data_df[data_df['token'] != 'usn'] #Harmony error
# data_df = data_df[data_df['protocol'] != 'shibaswap'] #Harmony error
data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar
data_df = data_df[~data_df['net_dollar_flow'].isna()] #50 bil error bar
# display(data_df[
# #         (data_df['protocol'] == 'sushiswap') & (data_df['chain'] == 'Harmony')
#         (data_df['net_dollar_flow'].notna()) & (data_df['net_dollar_flow'] != 0)
#         ].sort_values('net_dollar_flow'))


data_df.sort_values('net_dollar_flow')


In [ ]:
# data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')

# segment_cols = ['date','protocol','chain','token']

In [ ]:
netdf_df = data_df[['date','protocol','chain','net_dollar_flow']]
netdf_df = netdf_df.fillna(0)
# display(netdf_df)
netdf_df = netdf_df.groupby(['date','protocol','chain']).sum(['net_dollar_flow'])

netdf_df['cumul_net_dollar_flow'] = netdf_df.groupby(['protocol','chain']).cumsum()
netdf_df.reset_index(inplace=True)

# netdf_df[(netdf_df['protocol'] == 'uniswap') & (netdf_df['token'] != 'USDT')]

In [ ]:
summary_df = netdf_df[(netdf_df['date'].dt.date == date.today()-timedelta(days=1)) &\
                        (~netdf_df['chain'].str.contains('borrowed')) &\
                        (~netdf_df['chain'].str.contains('staking'))  
                        ]
summary_df = summary_df.sort_values(by='cumul_net_dollar_flow',ascending=False)
summary_df['direction'] = np.where(summary_df['cumul_net_dollar_flow']>=0,1,-1)
summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])
# display(summary_df)
fig = px.treemap(summary_df[summary_df['abs_cumul_net_dollar_flow'] !=0], \
                 path=[px.Constant("all"), 'chain', 'protocol'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values='abs_cumul_net_dollar_flow', color='direction'
                ,color_discrete_map={'(?)':'lightgrey', '0':'red', '1':'green'})
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.update_layout(tickprefix = '$')
fig.show()

In [ ]:
# netdf_df_plot = netdf_df[netdf_df['chain']=='Ethereum']

# fig = px.line(netdf_df_plot, x="date", y="net_dollar_flow", color="protocol", \
#              title="Daily Net Dollar Flow since Program Announcement",\
#             labels={
#                      "date": "Day",
#                      "net_dollar_flow": "Net Dollar Flow (N$F)"
#                  }
#             )
# fig.update_layout(
#     legend_title="App Name"
# )
# fig.update_layout(yaxis_tickprefix = '$')
# fig.show()

# # cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
# #              title="Cumulative Dollar Flow since Program Announcement",\
# #                    labels={
# #                      "date": "Day",
# #                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
# #                  }
# #             ,areamode='group')
# # cumul_fig.update_layout(yaxis_tickprefix = '$')
# # cumul_fig.show()


# cumul_fig = go.Figure()
# proto_names = netdf_df_plot['protocol'].drop_duplicates()
# print(proto_names)
# for p in proto_names:
#     cumul_fig.add_trace(go.Scatter(x=netdf_df_plot[netdf_df_plot['protocol'] == p]['date'] \
#                                    , y=netdf_df_plot[netdf_df_plot['protocol'] == p]['cumul_net_dollar_flow'] \
#                                     ,name = p\
#                                   ,fill='tozeroy')) # fill down to xaxis

# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.update_layout(
#     title="Cumulative Dollar Flow since Program Announcement",
#     xaxis_title="Day",
#     yaxis_title="Cumulative Net Dollar Flow (N$F)",
#     legend_title="App Name",
# #     color_discrete_map=px.colors.qualitative.G10
# )
# cumul_fig.show()